In [1]:
import pandas as pd
import requests
from simple_salesforce import Salesforce
from simple_salesforce.exceptions import SalesforceMalformedRequest

In [2]:
url = 'https://ctl-fiber.my.salesforce.com/services/oauth2/token'
params = {"grant_type": "password",
        "username":"qfqhuser_prod@lumen.com",
        "password":"QfQh2023Prod",
        "client_id":"3MVG9nkapUnZB56HQ6OpJKsXNoT64CpeLBR.cvpLL1nVP9RbG41P2_4f3mxPCoRIfA2Q2IsoCJctwQABgt9iK",
        "client_secret":"83FF3BBDB516839F6060A0C240BD545553E6BAAECFC5130F4C3F29420BC3A24D"
        }
r = requests.post(url,params=params)      
access_token1 = r.json().get("access_token")

sf=Salesforce(instance_url='https://ctl-fiber.my.salesforce.com',session_id=access_token1)


In [3]:
sf_item_query="""
    SELECT
    vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__AccountId__r.AccountName__c,
    vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__AccountId__r.AccountStatus__c,
    vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__OrderId__r.OrderNumber,
    vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__OrderId__r.vlocity_cmt__OrderStatus__c,
    vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationItemType__c,
    vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__c,
    Name,
    Id,
    vlocity_cmt__OrchestrationItemId__r.Name,
    vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__State__c,
    vlocity_cmt__OrchestrationItemId__r.CreatedDate
    FROM
        vlocity_cmt__OrchestrationDependency__c
    where
        vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__AccountId__r.Product_Name__c='Fiber Internet'
    and
        vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__OrderId__r.Type!='Change'
    and
        vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__AccountId__r.Service_State_Short__c not in ('NC','MO','TX','WI','AL','VA','OH','AR','PA','MS','GA','KS','TN','SC','IN','NJ','LA','IL')
    and 
        vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__OrderId__r.OrderNumber = '{customer_order}'
    LIMIT 100

"""


results = sf.query(sf_item_query)
re = results['records']

In [5]:
normalized_data = []
for record in re:
    normalized_record = {
        "Id": record.get("Id"),
        "AccountName": record.get("vlocity_cmt__OrchestrationItemId__r", {}).get("vlocity_cmt__OrchestrationPlanId__r", {}).get("vlocity_cmt__AccountId__r", {}).get("AccountName__c"),
        "AccountStatus": record.get("vlocity_cmt__OrchestrationItemId__r", {}).get("vlocity_cmt__OrchestrationPlanId__r", {}).get("vlocity_cmt__AccountId__r", {}).get("AccountStatus__c"),
        "OrderNumber": record.get("vlocity_cmt__OrchestrationItemId__r", {}).get("vlocity_cmt__OrchestrationPlanId__r", {}).get("vlocity_cmt__OrderId__r", {}).get("OrderNumber"),
        "OrderStatus": record.get("vlocity_cmt__OrchestrationItemId__r", {}).get("vlocity_cmt__OrchestrationPlanId__r", {}).get("vlocity_cmt__OrderId__r", {}).get("vlocity_cmt__OrderStatus__c"),
        "OrchestrationItemType": record.get("vlocity_cmt__OrchestrationItemId__r", {}).get("vlocity_cmt__OrchestrationItemType__c"),
        "OrchestrationPlanId": record.get("vlocity_cmt__OrchestrationItemId__r", {}).get("vlocity_cmt__OrchestrationPlanId__c"),
        "Name": record.get("Name"),
        "DependsOnItemId": record.get("vlocity_cmt__DependsOnItemId__c"),
        "OrchestrationDependencyDefinitionId": record.get("vlocity_cmt__OrchestrationDependencyDefinitionId__c"),
        "OrchestrationItemName": record.get("vlocity_cmt__OrchestrationItemId__r", {}).get("Name"),
        "OrchestrationItemState": record.get("vlocity_cmt__OrchestrationItemId__r", {}).get("vlocity_cmt__State__c"),
        "CreatedDate": record.get("vlocity_cmt__OrchestrationItemId__r", {}).get("CreatedDate")
    }
    normalized_data.append(normalized_record)
    
normalized_data

[]

In [4]:
df = pd.DataFrame(results['records'])
df =df.drop(columns = 'attributes', errors = 'ignore')
df.head()

,vlocity_cmt__OrchestrationItemId__r,vlocity_cmt__DependsOnItemId__c,vlocity_cmt__OrchestrationDependencyDefinitionId__c,Name,Id
0,{'attributes': {'type': 'vlocity_cmt__Orchestr...,a4nDo000002RKyGIAW,a4kDo0000006f3rIAA,Create Account & Contact Updates --> Device Se...,a4lDo000003ay0iIAA
1,{'attributes': {'type': 'vlocity_cmt__Orchestr...,a4nDo000002RKyNIAW,a4kDo000000dUA2IAM,HSI Fiber Installation Completed > Place HSI S...,a4lDo000003ay0jIAA
2,{'attributes': {'type': 'vlocity_cmt__Orchestr...,a4nDo000002RKyQIAW,a4kDo000000dUB0IAM,SOM Completed --> Place HSI Service in EULA WG,a4lDo000003ay0kIAA
3,{'attributes': {'type': 'vlocity_cmt__Orchestr...,a4nDo000002RKxyIAG,a4kDo00000120pzIAA,Create EULA WG Case --> Send EULA WG Reminder ...,a4lDo000003ay0lIAA
4,{'attributes': {'type': 'vlocity_cmt__Orchestr...,a4nDo000002RKxwIAG,a4kDo00000120pfIAA,Place HSI Service in EULA WG --> Create EULA W...,a4lDo000003ay0mIAA


In [5]:
df_1 = pd.json_normalize(df.to_dict(orient = 'records'))
df_1

,vlocity_cmt__DependsOnItemId__c,vlocity_cmt__OrchestrationDependencyDefinitionId__c,Name,Id,vlocity_cmt__OrchestrationItemId__r.attributes.type,vlocity_cmt__OrchestrationItemId__r.attributes.url,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.attributes.type,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.attributes.url,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__AccountId__r.attributes.type,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__AccountId__r.attributes.url,...,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__AccountId__r.AccountStatus__c,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__OrderId__r.attributes.type,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__OrderId__r.attributes.url,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__OrderId__r.OrderNumber,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__r.vlocity_cmt__OrderId__r.vlocity_cmt__OrderStatus__c,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationItemType__c,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__OrchestrationPlanId__c,vlocity_cmt__OrchestrationItemId__r.Name,vlocity_cmt__OrchestrationItemId__r.vlocity_cmt__State__c,vlocity_cmt__OrchestrationItemId__r.CreatedDate
0,a4nDo000002RKyGIAW,a4kDo0000006f3rIAA,Create Account & Contact Updates --> Device Se...,a4lDo000003ay0iIAA,vlocity_cmt__OrchestrationItem__c,/services/data/v52.0/sobjects/vlocity_cmt__Orc...,vlocity_cmt__OrchestrationPlan__c,/services/data/v52.0/sobjects/vlocity_cmt__Orc...,Account,/services/data/v52.0/sobjects/Account/001Do000...,...,Active,Order,/services/data/v52.0/sobjects/Order/801Do00000...,A00706991,Activated,Callout,a4pDo0000027LoSIAU,Device Selection callout to GCP,Completed,2024-04-02T16:33:42.000+0000
1,a4nDo000002RKyNIAW,a4kDo000000dUA2IAM,HSI Fiber Installation Completed > Place HSI S...,a4lDo000003ay0jIAA,vlocity_cmt__OrchestrationItem__c,/services/data/v52.0/sobjects/vlocity_cmt__Orc...,vlocity_cmt__OrchestrationPlan__c,/services/data/v52.0/sobjects/vlocity_cmt__Orc...,Account,/services/data/v52.0/sobjects/Account/001Do000...,...,Active,Order,/services/data/v52.0/sobjects/Order/801Do00000...,A00706991,Activated,Callout,a4pDo0000027LoSIAU,Place HSI Service in EULA WG,Skipped,2024-04-02T16:33:42.000+0000
2,a4nDo000002RKyQIAW,a4kDo000000dUB0IAM,SOM Completed --> Place HSI Service in EULA WG,a4lDo000003ay0kIAA,vlocity_cmt__OrchestrationItem__c,/services/data/v52.0/sobjects/vlocity_cmt__Orc...,vlocity_cmt__OrchestrationPlan__c,/services/data/v52.0/sobjects/vlocity_cmt__Orc...,Account,/services/data/v52.0/sobjects/Account/001Do000...,...,Active,Order,/services/data/v52.0/sobjects/Order/801Do00000...,A00706991,Activated,Callout,a4pDo0000027LoSIAU,Place HSI Service in EULA WG,Skipped,2024-04-02T16:33:42.000+0000
3,a4nDo000002RKxyIAG,a4kDo00000120pzIAA,Create EULA WG Case --> Send EULA WG Reminder ...,a4lDo000003ay0lIAA,vlocity_cmt__OrchestrationItem__c,/services/data/v52.0/sobjects/vlocity_cmt__Orc...,vlocity_cmt__OrchestrationPlan__c,/services/data/v52.0/sobjects/vlocity_cmt__Orc...,Account,/services/data/v52.0/sobjects/Account/001Do000...,...,Active,Order,/services/data/v52.0/sobjects/Order/801Do00000...,A00706991,Activated,Callout,a4pDo0000027LoSIAU,Send EULA WG Reminder Comm,Skipped,2024-04-02T16:33:42.000+0000
4,a4nDo000002RKxwIAG,a4kDo00000120pfIAA,Place HSI Service in EULA WG --> Create EULA W...,a4lDo000003ay0mIAA,vlocity_cmt__OrchestrationItem__c,/services/data/v52.0/sobjects/vlocity_cmt__Orc...,vlocity_cmt__OrchestrationPlan__c,/services/data/v52.0/sobjects/vlocity_cmt__Orc...,Account,/services/data/v52.0/sobjects/Account/001Do000...,...,Active,Order,/services/data/v52.0/sobjects/Order/801Do00000...,A00706991,Activated,AutoTask,a4pDo0000027LoSIAU,Create EULA WG Case,Skipp